In [12]:
# This is the notebook for python pandas challenge
# Using python 3.9.12

In [13]:
# Initial setup: grabbing the data into pandas dataframes
import pandas as pd
import os

# Get the absolute path of the directory containing the notebook
notebook_dir = os.path.abspath('') # On my machine, this returns /Users/galenmittermann/Documents/GitHub/pandas-challenge
print(notebook_dir)
# Then I set the path to the project directory
projectDir = os.path.dirname(notebook_dir)
# Then I create the file path relative to the project directory
schoolFilePath = os.path.join(projectDir, "PyCitySchools/Resources", "schools_complete.csv")
studentFilePath = os.path.join(projectDir, "PyCitySchools/Resources", "students_complete.csv")
# Now I read the csv file into a dataframe
schdf = pd.read_csv(schoolFilePath)
studf = pd.read_csv(studentFilePath)
# Look at the dataframes to make sure this worked
print(schdf.head())
print(studf.head())


/Users/galenmittermann/Documents/GitHub/pandas-challenge/PyCitySchools
   School ID            school_name      type  size   budget
0          0      Huang High School  District  2917  1910635
1          1   Figueroa High School  District  2949  1884411
2          2    Shelton High School   Charter  1761  1056600
3          3  Hernandez High School  District  4635  3022020
4          4    Griffin High School   Charter  1468   917500
   Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  

In [20]:
# Number of schools present: length of a list that contains the unique values in the School ID column
nUniqueSchools = len(schdf['School ID'].unique())
print(nUniqueSchools)

15


In [19]:
# Number of students: length of a list that contains unique values in the student_id column
nUniqueStudents = len(studf['Student ID'].unique())
print(nUniqueStudents)

39170


In [23]:
# Total budget is sum of the budget column
totalBudget = schdf['budget'].sum()
print(totalBudget)

24649428


In [26]:
# Average math score in the students is average of of the math_score column
avgMath = studf['math_score'].mean()
print(avgMath)

78.98537145774827


In [27]:
# Average reading score in the students is average of of the reading_score column
avgReading = studf['reading_score'].mean()
print(avgReading)

81.87784018381414
